## Dane

In [1]:
tekst_topic = ["I had a peanut butter sandwich for breakfast.",
                 "I like to eat almonds, peanuts and walnuts.",
                 "My neighbor got a little dog yesterday.",
                 "Cats and dogs are mortal enemies.",
                 "You mustn’t feed peanuts to your dog."]

## Przygotowanie tekstu

In [2]:
import re
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")

stopwords = ["a", "able", "about", "across", "after", "all", "almost", "also", "am", "among", "an", "and", "any", "are", "as", "at", "be", "because", "been", "but", "by", "can", "cannot", "could", "dear", "did", "do", "does", "either", "else", "ever", "every", "for", "from", "get", "got", "had", "has", "have", "he", "her", "hers", "him", "his", "how", "however", "i", "if", "in", "into", "is", "it", "its", "just", "least", "let", "like", "likely", "may", "me", "might", "most", "must", "my", "neither", "no", "nor", "not", "of", "off", "often", "on", "only", "or", "other", "our", "own", "rather", "said", "say", "says", "she", "should", "since", "so", "some", "than", "that", "the", "their", "them", "then", "there", "these", "they", "this", "tis", "to", "too", "twas", "us", "wants", "was", "we", "were", "what", "when", "where", "which", "while", "who", "whom", "why", "will", "with", "would", "yet", "you", "your"]

def czysc_tekst(tekst):
    
    temp = re.sub("\s{2,}", " ", tekst)
    temp = re.sub("(\r\n|\r|\n)", " ", temp) 
    temp = temp.lower() 
    temp = re.sub("[!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]", "", temp) 
    temp = re.sub("\d", "", temp) 
    temp = re.sub("\s{2,}", " ", temp) 
    temp = temp.strip()
    return temp

czysty = [czysc_tekst(el) for el in tekst_topic]
tokeny = [re.split("\s", el) for el in czysty]
termy = [[t for t in el if t not in stopwords] for el in tokeny]
termy_stem = [[stemmer.stem(t) for t in el] for el in termy]

In [3]:
termy_stem

[['peanut', 'butter', 'sandwich', 'breakfast'],
 ['eat', 'almond', 'peanut', 'walnut'],
 ['neighbor', 'littl', 'dog', 'yesterday'],
 ['cat', 'dog', 'mortal', 'enemi'],
 ["mustn't", 'feed', 'peanut', 'dog']]

## Analiza ukrytych grup semantycznych

**Rozkład według wartości osobliwych** (**SVD** *Singular Value Decomposition*) to pewien rozkład (dekompozycja) macierzy na iloczyn trzech specyficznych macierzy.

### Macierz dokument term

In [4]:
import numpy as np
# import scipy

In [5]:
m_Tf = np.array([[0, 1, 1, 0, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  1,  1,  0,  0],
[1, 0, 0, 0, 0, 1, 0, 0, 0,  1,  0,  0,  0,  0,  1,  0,  1,  0],
[0, 0, 0, 0, 1, 0, 0, 0, 1,  0,  1,  0,  0,  1,  0,  0,  0,  1],
[0, 0, 0, 1, 1, 0, 1, 0, 0,  0,  0,  1,  0,  0,  0,  0,  0,  0],
[0, 0, 0, 0, 1, 0, 0, 1, 0,  0,  0,  0,  1,  0,  1,  0,  0,  0]])
termy = np.array(["almonds", "breakfast", "butter",  "cat",   "dog",   "eat",   "enemies", "feed",  "got", 
"like",  "littl",   "mortal",  "mustn’t", "neighbor"  "peanut",  "sandwich"  "walnuts", "yesterday"])

In [6]:
termy_topic, waznosc, dokument_topic = np.linalg.svd(m_Tf.transpose())

dtm = (termy_topic[:, :5] @ np.diag(waznosc) @ dokument_topic).transpose()
print(dtm.round())

[[-0.  1.  1. -0. -0. -0. -0. -0.  0. -0.  0. -0. -0.  0.  1.  1. -0.  0.]
 [ 1.  0.  0. -0. -0.  1. -0. -0. -0.  1. -0. -0. -0. -0.  1.  0.  1. -0.]
 [ 0.  0.  0. -0.  1.  0. -0. -0.  1.  0.  1. -0. -0.  1. -0.  0.  0.  1.]
 [-0.  0.  0.  1.  1. -0.  1. -0. -0. -0. -0.  1. -0. -0. -0.  0. -0. -0.]
 [-0. -0. -0. -0.  1. -0. -0.  1. -0. -0. -0. -0.  1. -0.  1. -0. -0. -0.]]


In [7]:
np.diag(waznosc)

array([[2.6335209 , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 2.37023923, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.88079812, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.83901224, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.58970629]])

In [8]:
termy_topic

array([[-1.78452839e-01, -2.53772256e-01, -2.71180784e-01,
         2.02145492e-01,  8.67109701e-02, -3.25197498e-01,
         1.44780731e-03, -1.04049547e-02,  1.15160361e-01,
        -3.25197498e-01,  1.15160361e-01,  1.44780731e-03,
        -1.04049547e-02,  1.15160361e-01, -5.89103091e-01,
        -2.53500638e-01, -3.25197498e-01,  1.15160361e-01],
       [-1.33107671e-01, -1.56839879e-01,  2.33434006e-01,
        -3.27078277e-01,  2.70096655e-01,  3.02707716e-01,
         5.16632111e-03, -3.71287926e-02,  2.15576023e-01,
         3.02707716e-01,  2.15576023e-01,  5.16632111e-03,
        -3.71287926e-02,  2.15576023e-01, -2.02169665e-01,
        -4.67748589e-01,  3.02707716e-01,  2.15576023e-01],
       [-1.33107671e-01, -1.56839879e-01,  2.33434006e-01,
        -3.27078277e-01,  2.70096655e-01, -8.35237482e-02,
        -2.56187161e-02,  1.84113991e-01, -3.74539398e-01,
        -8.35237482e-02, -3.74539398e-01, -2.56187161e-02,
         1.84113991e-01, -3.74539398e-01, -1.32795444e

### Zadanie

Napisz program, który wyświetli następującą inforamcję:

```
Z wątkiem {nr} związane sa termy: {... należy wypisać wszystkie termy związane z danym wątkiem}

```

In [26]:
#Numer wątku
nr = 2

related_terms = [termy[i] for i in range(len(termy)) if m_Tf[nr, i] != 0]
related_terms_str = ', '.join(related_terms)

print(f"Z wątkiem {nr} związane są termy: {related_terms_str}")


Z wątkiem 2 związane są termy: dog, got, littl, neighborpeanut


## Analiza wątków z biblioteką `gensim`

*Topic models* jest rozszerzeniem klasycznych metod w przetwarzana języka naturalnego, takich jak *unigram model* i *Latent Semantic Analysis*. w poszukiwaniu tematu (wątku) dokumentu wykorzystuje się jedną z metod redukcji wymiaru przestrzeni cech: **latent Dirichlet allocation** (**LDA**) - to statystyczny model, który opisuje (hipotetyczny) proces losowy generujący kolejne słowa każdego dokumentu zgodnie z rozkładem tematów w dokumencie.

In [ ]:
# !pip install smart_open==1.9.0
# !pip install gensim==3.4.0

In [11]:
import smart_open
import gensim

In [12]:
id_term = gensim.corpora.Dictionary(termy_stem)

for i, k in id_term.items():
    print(i, k)

0 breakfast
1 butter
2 peanut
3 sandwich
4 almond
5 eat
6 walnut
7 dog
8 littl
9 neighbor
10 yesterday
11 cat
12 enemi
13 mortal
14 feed
15 mustn't


In [13]:
bow_topic = [id_term.doc2bow(term) for term in termy_stem]

In [14]:
print(bow_topic)
[(id_term[idterm], licz) for idterm, licz in bow_topic[0]]

[[(0, 1), (1, 1), (2, 1), (3, 1)], [(2, 1), (4, 1), (5, 1), (6, 1)], [(7, 1), (8, 1), (9, 1), (10, 1)], [(7, 1), (11, 1), (12, 1), (13, 1)], [(2, 1), (7, 1), (14, 1), (15, 1)]]


[('breakfast', 1), ('butter', 1), ('peanut', 1), ('sandwich', 1)]

In [15]:
liczba_watkow = 3

lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_topic,
                                           id2word=id_term,
                                           num_topics=liczba_watkow,                                            
                                           alpha='symmetric')

In [16]:
liczba_termow = 5

lda_model.print_topics(num_words = liczba_termow)

#Interpretacja: większy współczynnik oznacza że term jest ważniejszy (np. w 2 termie słowo "peanut" jest ważniejsze niż "sandwich")

[(0,
  '0.141*"peanut" + 0.136*"walnut" + 0.136*"almond" + 0.135*"eat" + 0.042*"dog"'),
 (1,
  '0.134*"peanut" + 0.133*"dog" + 0.077*"sandwich" + 0.076*"feed" + 0.076*"littl"'),
 (2,
  '0.141*"dog" + 0.136*"enemi" + 0.136*"mortal" + 0.136*"cat" + 0.040*"peanut"')]

## Wizualizacja analizy wątków z biblioteką `pyLDAvis`

In [ ]:
# !pip install pyLDAvis

In [19]:
import pyLDAvis
import pyLDAvis.gensim

In [20]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_topic, id_term)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.009927 -0.064761       1        1  54.491784
2     -0.063341  0.024121       2        1  22.754531
0      0.053414  0.040640       3        1  22.753685, topic_info=        Term      Freq     Total Category  logprob  loglift
6     walnut  1.000000  1.000000  Default  16.0000  16.0000
12     enemi  1.000000  1.000000  Default  15.0000  15.0000
13    mortal  1.000000  1.000000  Default  14.0000  14.0000
4     almond  1.000000  1.000000  Default  13.0000  13.0000
11       cat  1.000000  1.000000  Default  12.0000  12.0000
..       ...       ...       ...      ...      ...      ...
15   mustn't  0.169135  1.168506   Topic3  -3.2923  -0.4523
8      littl  0.169006  1.168691   Topic3  -3.2931  -0.4533
14      feed  0.167371  1.168835   Topic3  -3.3028  -0.4631
3   sandwich  0.167249  1.169942   Topic3  -3.3036  -0.4648
7        dog  0.191421  2.284076   Topic3  -3.1686  -0.9988

[64 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
4         3  0.986570     almond
0         1  0.855846  breakfast
1         1  0.855668     butter
11        2  0.985407        cat
7         1  0.437814        dog
7         2  0.437814        dog
5         3  0.985688        eat
12        2  0.985266      enemi
14        1  0.855553       feed
8         1  0.855658      littl
13        2  0.985961     mortal
15        1  0.855794    mustn't
9         1  0.856290   neighbor
2         1  0.437439     peanut
2         3  0.437439     peanut
3         1  0.854743   sandwich
6         3  0.987336     walnut
10        1  0.859096  yesterday, R=16, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])